In [36]:
import requests  # Библиотека для запроса
import re as stringTOfloat     # модуль для нахождения чисел внутри строки !!!!!!!!! Более внимательнее рассомтреть
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import geopandas as gpd
from shapely.geometry import Point, LineString
import flexpolyline
import geojson
import folium
from scipy.ndimage import gaussian_filter1d
import itertools
from scipy.spatial import distance
from pprint import pprint
import time

### Первичные данные

In [37]:
GPS_SIGNAL_TEST_VALUE = (47.552556,42.957140)
DISTANCE_VALUE_TEST = 0.01000

In [38]:
def select_segment_on_tuple(tuple_coord):
    return all_segments_on_coords[tuple_coord[0]][tuple_coord[1]]

# Написать на основе данного кода функцию для взятия данных о координатах из json, 


study library geometry method multilinestring


In [39]:
def json_coord_to_tuple(string):
    intermediate_list = list(map(float, stringTOfloat.findall(r"""\d+ # one or more digits
                          (?: # followed by...
                              \. # a decimal point 
                              \d+ # and another set of one or more digits
                          )? # zero or one times""",
                                      string,
                                      stringTOfloat.VERBOSE)))
    return coords_list_transformator_coords_tuple(intermediate_list)

#  Функция для создания из координат кортежей 

In [40]:
def coords_list_transformator_coords_tuple(coords_list):
    buf = list()
    for i in range(0, len(coords_list), 2):
        buf.append((coords_list[i], coords_list[i + 1]))
    return buf

#  Функция для создания из формирования запроса

In [41]:
def inquiry_json_from_Here(layers_here_inquiry, grid_map_inquiry, api_key_here_inquiry):
    # Выбор запращ=шиваемых слоев
    layers_here = f'layers={layers_here_inquiry}&levels=13'
    
    # Зона отображения
    grid_map = f'in=bbox:{grid_map_inquiry}'
    
    # API
    api_key_here = f'apiKey={api_key_here_inquiry}'
    
    # Сформированный запрос
    inquiry = f'https://smap.hereapi.com/v8/maps/attributes?{layers_here}&{grid_map}&{api_key_here}&meta=1'
    
    response_to_request = requests.get(inquiry).json()
    
    return response_to_request

# Запрос на сервер.
Ответ представляет собой словарь с данными с карты. Скорее всего придется повторять запрос для увеличения площади отрисовки
## на всякий случай лучше не удалять данные для дальнейшей обработки

### grid_map участок краты прямоугольной формы.
X_center, Y_center coords on GPS signal

grid_map = X_center - 0.025000, Y_center -0,001000, X_center + 0.025000, Y_center -0,001000

проверить координатную сетку

0.01 == 1km

# Функция для расчета grid_map на основе введенной дистанции и точки от спутника
Ответ формирутся в виде СПИСКА с координатами

In [42]:
def generate_grid_map_depending_on_distance(distance, gps):
    x_center, y_center = gps
    x_0 = list(map(lambda x: round(x + x_center, 6), np.arange((-distance / 2), (distance / 2) + 0.0053, 0.0053)))
    y_0 = list(map(lambda y: round(y + y_center, 6), np.arange((-distance / 2), (distance / 2) + 0.0023, 0.0023)))
    grid_map_depending = list()
    for i in range(len(x_0) - 1):
        for j in range(len(y_0) - 1):
            grid_map_depending.append((f'{x_0[i]},{y_0[j]},{x_0[i + 1]},{y_0[j + 1]}'))
    return grid_map_depending

## Потом доработать чтобы отсортировать по названиям улиц
for i in range(len(all_here_map_coords)):
    for j in range(len(all_here_map_coords[0]['geometries'])):
        print(all_here_map_coords[i]['geometries'][j]['attributes']['NAME'])
### Примерный код

In [43]:
layers_here = 'ROAD_GEOM_FC5,ROAD_GEOM_FC4,ROAD_GEOM_FC3,ROAD_GEOM_FC2,ROAD_GEOM_FC1' #ROAD_GEOM_FC5,
api_key_here = 'FM-SKdcDUgjKRgM_EuazV2yvylQWDJKWljnDO1mbtn4'
all_here_map_coords = list(map(lambda x: inquiry_json_from_Here(layers_here, x, api_key_here), generate_grid_map_depending_on_distance(DISTANCE_VALUE_TEST, GPS_SIGNAL_TEST_VALUE)))

# Код для заполнения координатами переменной

In [44]:
all_coords = list()
for i in range(len(all_here_map_coords)):
    for j in range(len(all_here_map_coords[i]['geometries'])):
        all_coords.append(json_coord_to_tuple(all_here_map_coords[i]['geometries'][j]['geometry']))
pprint(all_coords)

[[(47.55276, 42.95357), (47.55315, 42.95375)],
 [(47.55269, 42.95354), (47.55276, 42.95357)],
 [(47.55087, 42.95431), (47.5505, 42.95455), (47.55012, 42.95448)],
 [(47.54903, 42.95394),
  (47.54888, 42.95395),
  (47.54813, 42.95405),
  (47.54773, 42.95422),
  (47.5472, 42.95436),
  (47.54903, 42.95394),
  (47.54888, 42.95395),
  (47.54813, 42.95405),
  (47.54773, 42.95422),
  (47.5472, 42.95436)],
 [(47.54903, 42.95394),
  (47.54911, 42.95397),
  (47.54924, 42.95403),
  (47.54948, 42.95414)],
 [(47.55028, 42.95377),
  (47.5505, 42.95397),
  (47.55049, 42.95402),
  (47.55046, 42.95407),
  (47.55034, 42.95418),
  (47.55022, 42.95432),
  (47.55012, 42.95448)],
 [(47.54889, 42.95346),
  (47.54881, 42.95342),
  (47.54861, 42.95341),
  (47.54844, 42.95341),
  (47.54807, 42.95357),
  (47.54889, 42.95346),
  (47.54881, 42.95342),
  (47.54861, 42.95341),
  (47.54844, 42.95341),
  (47.54807, 42.95357)],
 [(47.54889, 42.95346),
  (47.54896, 42.95354),
  (47.54901, 42.95364),
  (47.54903, 42.95378

#  Код для построения отрезков на основании координат

In [45]:
%%time
all_segments_on_coords = []
for i in all_coords:
    temporary = []
    for j in range(len(i) - 1):
        temporary.append((LineString([ i[j], i[j + 1] ])))
    all_segments_on_coords.append(temporary)

Wall time: 16.3 ms


In [59]:
all_segments_on_coords

[[<shapely.geometry.linestring.LineString at 0x1d3f4ca8040>],

In [46]:
%%time
all_segments_intersection = {}
for position_in_primary, value_from_primary in enumerate(all_segments_on_coords):
    temporary_value = []
    for position_in_secondary, value_from_secondary in enumerate(value_from_primary):
        for position_in_primary_additional, value_from_primary_additional in enumerate(all_segments_on_coords):
            for position_in_secondary_additional, value_from_secondary_additional in enumerate(value_from_primary_additional):
                if (type(value_from_secondary.intersection(value_from_secondary_additional)) is Point) and value_from_secondary.intersection(value_from_secondary_additional) and position_in_primary != position_in_primary_additional:
                    temporary_value.append((value_from_secondary.intersection(value_from_secondary_additional), (position_in_primary, position_in_secondary), (position_in_primary_additional, position_in_secondary_additional)))    
    all_segments_intersection[str(position_in_primary)] = temporary_value

Wall time: 58.6 s


In [60]:
all_segments_intersection

{'0': [(<shapely.geometry.point.Point at 0x1d3f4df77c0>, (0, 0), (1, 0)),
  (<shapely.geometry.point.Point at 0x1d3f4df7820>, (0, 0), (8, 0)),
  (<shapely.geometry.point.Point at 0x1d3f4df75e0>, (0, 0), (202, 0)),
  (<shapely.geometry.point.Point at 0x1d3f4df7640>, (0, 0), (205, 0))],
 '1': [(<shapely.geometry.point.Point at 0x1d3f4df76a0>, (1, 0), (0, 0)),
  (<shapely.geometry.point.Point at 0x1d3f4df7520>, (1, 0), (12, 0)),
  (<shapely.geometry.point.Point at 0x1d3f4df7580>, (1, 0), (14, 0)),
  (<shapely.geometry.point.Point at 0x1d3f4df7460>, (1, 0), (36, 0)),
  (<shapely.geometry.point.Point at 0x1d3f4df74c0>, (1, 0), (208, 0))],
 '2': [(<shapely.geometry.point.Point at 0x1d3f4df7340>, (2, 0), (17, 1)),
  (<shapely.geometry.point.Point at 0x1d3f4df73a0>, (2, 0), (21, 1)),
  (<shapely.geometry.point.Point at 0x1d3f4df7400>, (2, 1), (5, 5)),
  (<shapely.geometry.point.Point at 0x1d3f4df7100>, (2, 1), (21, 0)),
  (<shapely.geometry.point.Point at 0x1d3f4df7160>, (2, 1), (23, 5)),
  (<

In [47]:
%%time
all_segments_intersection_new = {}
all_segment = []
for position_in_primary, value_from_primary in enumerate(all_segments_on_coords):
    temporary_value = []
    for position_in_secondary, value_from_secondary in enumerate(value_from_primary):
        for position_in_primary_additional, value_from_primary_additional in enumerate(all_segments_on_coords):
            for position_in_secondary_additional, value_from_secondary_additional in enumerate(value_from_primary_additional):
                if (type(value_from_secondary.intersection(value_from_secondary_additional)) is Point) and value_from_secondary.intersection(value_from_secondary_additional) and position_in_primary != position_in_primary_additional:
                    all_segment.append(json_coord_to_tuple(str(value_from_secondary.intersection(value_from_secondary_additional))))

Wall time: 1min 5s


In [61]:
all_segment

[[(47.55276, 42.95357)],
 [(47.55315, 42.95375)],
 [(47.55315, 42.95375)],
 [(47.55315, 42.95375)],
 [(47.55276, 42.95357)],
 [(47.55269, 42.95354)],
 [(47.55269, 42.95354)],
 [(47.55269, 42.95354)],
 [(47.55276, 42.95357)],
 [(47.55087, 42.95431)],
 [(47.5505, 42.95455)],
 [(47.55012, 42.95448)],
 [(47.5505, 42.95455)],
 [(47.55012, 42.95448)],
 [(47.55012, 42.95448)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.54903, 42.95394)],
 [(47.5505, 42.95397)],
 [(47.5505, 42.95397)],
 [(47.55049, 42.95402)],
 [(47.55049, 42.95402)],
 [(47.55046, 42.95407)],
 [(47.55046, 42.95407)],
 [(47.55034, 42.95418)],
 [(47.55034, 42.95418)],
 [(47.55022, 42.95432)],
 [(47.55012, 42.95448)],
 [(47.55012, 42.95448)],
 [(47.55022, 42.95432)],
 [(47.55012, 42.95448)],
 [(47.54889, 42.95346)],
 [(47.54889, 42.95346)],
 [(4

In [ ]:
x = list()
for i in range(len(all_segment)):
    x.append(all_segment[i][0][0])

In [ ]:
y = list()
for i in range(len(all_segment)):
    y.append(all_segment[i][0][1])

In [ ]:
x_1 = np.array([i for i in x])
y_1 = np.array([i for i in y])

In [ ]:
plt.plot(x_1, y_1, 'ro')

In [54]:
x_original_point = 4755255.6
y_original_point = 4295714.0

mx = x_1 * 100000
my = y_1 * 100000

mxy = list(zip(mx,my))

picture_df = gpd.GeoDataFrame(
    {'id': range(0, len(mxy))}, 
    crs="EPSG:3857", 
    geometry=[Point(resu) for resu in mxy]
)

picture_df['geometry'] = picture_df['geometry'].to_crs(epsg=4326)

picture_df.to_file("cat.geojson", driver='GeoJSON', encoding="utf-8")

# Формируем список с нумерацией всех улиц
street_dict = {}
for i in range(len(all_coords)):
    street_dict[str(i)] = all_coords[i]

In [72]:
street_ruslan_t = {}
for number_street in street_ruslan:
    for all_coords_of_number_street in street_ruslan[number_street]:
        for i in street_ruslan:

            if i == number_street:
                continue
            for position, coords in enumerate(street_ruslan[i]):
                #print(position)
                if coords == all_coords_of_number_street:
                    print(number_street, i, position, coords)
    

NameError: name 'street_ruslan' is not defined

In [ ]:
# Формируем словарь вида 'номер улицы' : 'список пересекаемых номеров улиц'
street_dict_t = {}
counter = 0
for i in range(len(street_dict)):
    test = street_dict[str(i)]    # перебираю все улицы, их координаты
    street_dict_t[str(i)] = []
    counter += 1
    for j in range(len(test)//2):    # перебираю пары координат для каждой улицы (x, y)
        for item in street_dict.items():     # перебираем повторно все улицы, для сравнения с целевой
            if item[0] != str(i):   # проверяем что повторно взятая улица не совпадает с первоначальной
                for k in range(len(item[1])//2):       # перебираем пары координат для повторно взятой улицы
                    if [test[j*2], test[j*2+1]] == [item[1][k*2], item[1][k*2+1]]:   # проверяем координаты целевой и повторно взятой улиц
                        street_dict_t[str(i)] == street_dict_t[str(i)].append({item[0]: [item[1][k*2], item[1][k*2+1]]})
street_dict_of_point_coin = {}
for street, coincidence_coords_on_street in street_dict_t.items():
    x = {}
    for i in coincidence_coords_on_street:
        for j, k in i.items():
            x[j] = k
    street_dict_of_point_coin[street] = x
#street_dict_of_point_coin
street_dict_t
#print(len([x for x in street_dict_t.items() if len(x[1]) > 1]))

In [ ]:
street_select = '0'

street_coin_of_select_street = list(map(lambda x: x, street_dict_of_point_coin[street_select].keys())) # Улицы которые пересекаются в выбранной улицой.
for i in street_coin_of_select_street:
    coord_1 = street_dict_of_point_coin[street_select][i]
    print(coord_1)
    

In [ ]:
street_select = '0'

street_coin_of_select_street = list(map(lambda x: x, street_dict_of_point_coin[street_select].keys())) # Улицы которые пересекаются в выбранной улицой.
for i in street_coin_of_select_street:
    print(type(i))
    for j in street_coin_of_select_street[i[0]]:
        print(j)

In [ ]:
Segment((1, 0),(0, 1))


In [ ]:
for i in street_0:
    coord_1 = coincidence_coords(i[0][0], street_select)
    coord_2 = i[0][0]
    print(coord_1, ' - ', coord_2)

#street_dict_t

In [ ]:
def street_angle(coord1, coord2, coord3):
    ba = np.array(coord1) - np.array(coord2)
    bc = np.array(coord2) - np.array(coord3)

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

In [157]:
test_coords = [[0,0], [0, 1], [1, 2], [5, 1]]

d = distance.euclidean(tuple(test_coords[0]), tuple(test_coords[1]))

In [160]:
for i in range(len(test_coords) - 1):
    point1 = test_coords[i]
    point2 = test_coords[i + 1]
    if i == len(test_coords) - 2:
        point3 = test_coords[0]
    else:
        point3 = test_coords[i + 2]
    dist1 = distance.euclidean(tuple(point1), tuple(point2))
    dist2 = distance.euclidean(tuple(point2), tuple(point3))

    angle = street_angle(point1, point2, point3)
    
    print(angle)
    print(point1)
    print(tuple(point1))

45.00000000000001
[0, 0]
(0, 0)
59.03624346792648
[0, 1]
(0, 1)
154.65382405805332
[1, 2]
(1, 2)


In [ ]:
def json_coord_to_float_(string):
    return list(map(float, stringTOfloat.findall(r"""\d+ # one or more digits
                          (?: # followed by...
                              \. # a decimal point 
                              \d+ # and another set of one or more digits
                          )? # zero or one times""",
                                      string,
                                      stringTOfloat.VERBOSE)))

In [143]:
all_coords_ = list()
for i in range(len(all_here_map_coords)):
    for j in range(len(all_here_map_coords[i]['geometries'])):
        all_coords_.append(json_coord_to_float_(all_here_map_coords[i]['geometries'][j]['geometry']))

In [1]:
x_buf = list(map(lambda x: x[::2], all_coords_))
y_buf = list(map(lambda y: y[1::2], all_coords_))
x_chain = list(itertools.chain(*x_buf))
y_chain = list(itertools.chain(*y_buf))
xd = (set(zip(x_chain, y_chain)))
xs = [i[0] for i in xd]
ys = [i[1] for i in xd]
matrix = np.array([xs, ys])
print(all_coords_)

NameError: name 'all_coords_' is not defined

In [ ]:
plt.plot(matrix[0], matrix[1],  'o')

### https://developer.here.com/documentation/advanced-datasets-api/dev_guide/topics/here-map-content.html